# Задание 5.

Загрузите [данные по изменению температуры поверхности земли](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data). Для этого может понадобится зарегистрироваться на [Kaggle](https://kaggle.com). Затем нужно будет работать с данными, которые содержатся в файле **GlobalLandTemperaturesByMajorCity.csv**

**NB** Все подсчеты необходимо делать с помощью `PySpark`, без применения `pandas api`. Можно использоать `SQL`.

In [1]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("PySpark") \
    .getOrCreate()

pandas_df = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv')
df = spark.createDataFrame(pandas_df)

df.show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 16:09:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/03 16:09:35 WARN TaskSetManager: Stage 0 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.


+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1849-01-01|            26.704|                        1.435|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-02-01|            27.434|                        1.362|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-03-01|            28.101|                        1.612|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-04-01|             26.14|           1.3869999999999998|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-05-01|            25.427|                          1.2|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-06-01|            24.844|                        1.402|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1849-07-01|24.058000000000003|                        1.254|Abidjan|Côte

# Задание 4.1 (1 балл)

В последующих заданиях будут учитываться данные начиная с 01.01.1950. Для этого создайте новый `DataFrame`, в котором удалены все строки до 01.01.1950. Используйте созданный DataFrame в последующих заданиях.  

In [2]:
clear_df = df.filter(col('dt') >= '1950-01-01').filter(col('AverageTemperature').isNotNull())
clear_df.createOrReplaceGlobalTempView("T")
clear_df.show(10)
clear_df = clear_df.dropna()

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1950-01-01|26.773000000000003|                        0.239|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-02-01|27.526999999999997|                        0.348|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-03-01|            28.344|                        0.431|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-04-01|             27.83|                        0.467|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-05-01|            26.896|                        0.248|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-06-01|            25.454|                        0.209|Abidjan|Côte D'Ivoire|   5.63N|    3.23W|
|1950-07-01|            24.878|                        0.403|Abidjan|Côte

24/07/03 16:09:43 WARN TaskSetManager: Stage 1 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.
24/07/03 16:09:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Задание 4.2 (2 балла)

Найдите город, для которого выборочная дисперсия температур на приведенных данных максимальна. 

In [8]:
from pyspark.sql.functions import variance

variance_data = clear_df.groupBy("City").agg(variance(col("AverageTemperature")).alias("TemperatureVariance"))
variance_data.orderBy(col("TemperatureVariance").desc()).first()

24/07/03 16:12:45 WARN TaskSetManager: Stage 8 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.


Row(City='Harbin', TemperatureVariance=218.898615951821)

# Задание 4.3 (2 баллов)

Посчитайте данные по среднегодовой температуре в Санкт-Петербурге. Определите года, в которых средняя температура была выше, чем в предыдущем  и последующем году. 

In [19]:
from pyspark.sql.functions import year, avg, month
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead

spb_data = clear_df.filter(col('City') == 'Saint Petersburg')
spb_data = spb_data.withColumn('Year', year(col('dt')))
avg_annual_temp_spb = spb_data.groupBy('Year').agg(avg(col('AverageTemperature')).alias('YeaAverageTemperature'))

avg_annual_temp_spb = avg_annual_temp_spb.orderBy('Year')

window_spec = Window.orderBy("Year")
avg_annual_temp_spb = avg_annual_temp_spb.withColumn("PrevYearTemp", lag("YeaAverageTemperature").over(window_spec))
avg_annual_temp_spb = avg_annual_temp_spb.withColumn("NextYearTemp", lead("YeaAverageTemperature").over(window_spec))
result = avg_annual_temp_spb.filter(
    (col("YeaAverageTemperature") > col("PrevYearTemp")) & (col("YeaAverageTemperature") > col("NextYearTemp"))
)

result.select("Year").collect()


24/07/03 16:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 16:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 16:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 16:17:13 WARN TaskSetManager: Stage 65 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.
24/07/03 16:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 16:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 16:17:13 WARN WindowExec

[Row(Year=1953),
 Row(Year=1957),
 Row(Year=1959),
 Row(Year=1961),
 Row(Year=1964),
 Row(Year=1967),
 Row(Year=1972),
 Row(Year=1975),
 Row(Year=1977),
 Row(Year=1979),
 Row(Year=1983),
 Row(Year=1986),
 Row(Year=1989),
 Row(Year=1992),
 Row(Year=1995),
 Row(Year=1997),
 Row(Year=2000),
 Row(Year=2002),
 Row(Year=2005),
 Row(Year=2008),
 Row(Year=2011)]

# Задание 4.4 (4 балла)

Найдите города, для которых: 
1. Разница между максимальным и минимальным значением среднегодовой температуры в выборке максимальна.
2. Самая большая средняя разница между средней температурой января и средней температурой июля.
3. Наибольшее среднее количество месяцев с отрицательной температурой в году.

Максимальная разница между максимальной и минимальной среднегодовой температурой



In [17]:
city_year_temp = clear_df.withColumn('Year', year(col('dt'))).groupBy('City', 'Year').agg(avg(col('AverageTemperature')).alias('YeaAverageTemperature'))
city_year_temp.createOrReplaceTempView('T2')
city_temp_range_sql = """select City, min(YeaAverageTemperature) as min_temp, max(YeaAverageTemperature) as max_temp from T2 group by City"""
city_temp_range = spark.sql(city_temp_range_sql)
city_temp_range = city_temp_range.withColumn('temp_diff', col('max_temp') - col('min_temp'))
city_temp_range.orderBy(col('temp_diff').desc()).first()

24/07/03 16:14:48 WARN TaskSetManager: Stage 59 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.


Row(City='Mashhad', min_temp=10.6885, max_temp=15.9385, temp_diff=5.25)

Самая большая средняя разница между средней температурой января и средней температурой июля.

In [22]:
january_temp = clear_df.filter(month(col('dt')) == 1).groupBy('City').agg(avg(col('AverageTemperature')).alias('jan_avg'))
july_temp = clear_df.filter(month(col('dt')) == 7).groupBy('City').agg(avg(col('AverageTemperature')).alias('jul_avg'))

city_jan_jul_diff = january_temp.join(july_temp, on="City").withColumn("jan_jul_diff", col("jul_avg") - col("jan_avg"))
city_jan_jul_diff.orderBy(col("jan_jul_diff").desc()).first()

24/07/03 16:17:34 WARN TaskSetManager: Stage 89 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.
24/07/03 16:17:34 WARN TaskSetManager: Stage 90 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.


Row(City='Harbin', jan_avg=-18.578421875000004, jul_avg=23.414296874999998, jan_jul_diff=41.99271875)

Наибольшее среднее количество месяцев с отрицательной температурой в году.  

In [25]:
from pyspark.sql.functions import count_distinct
clear_df = clear_df.withColumn('Month', month(col('dt')))
clear_df = clear_df.withColumn('Year', year(col('dt')))

negative_temp_months = clear_df.filter(col('AverageTemperature') < 0).groupBy('City', 'Year').agg(count_distinct(col('Month')).alias('neg_temp_months'))
avg_neg_temp_months_city = negative_temp_months.groupBy('City').agg(avg(col('neg_temp_months')).alias('avg_neg_temp_months'))

avg_neg_temp_months_city.orderBy(col('avg_neg_temp_months').desc()).first()

24/07/03 16:20:32 WARN TaskSetManager: Stage 95 contains a task of very large size (1003 KiB). The maximum recommended task size is 1000 KiB.


Row(City='Harbin', avg_neg_temp_months=4.90625)